In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

from sklearn.preprocessing import MinMaxScaler

/Users/junginseo/opt/anaconda3/envs/dl_env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('concat.csv', index_col=0)

In [3]:
df.drop(df.columns[-5:], axis = 1, inplace=True)

In [4]:
df.shape

(11044005, 8)

데이터 갯수 조건에 맞지 않는 Ticker 선별

1. 데이터 갯수가 4년치 미만인 Ticker는 제거

In [5]:
del_ticker_list = df['Ticker'].value_counts()[df['Ticker'].value_counts()<=252*4].index

In [6]:
del_ticker_index = df[df['Ticker'].isin(del_ticker_list)].index

In [7]:
df_new = df.drop(del_ticker_index) 

In [8]:
df_new.shape

(10221509, 8)

In [9]:
# 강사님의 윈도우 코드 복붙(이건 우리 입맛에 나중에 바꿔도 상관없음)
def my_window_data(df, target, window_size=20):
    X_list = []
    y_list = []
    for i in range(len(df) - window_size-1):
        X = df.iloc[i:i+window_size]
        y = df.iloc[i+window_size][target]
        X_list.append(np.array(X))      #[[] ,[] ,[] ,[]]
        y_list.append(np.array(y))       #[]   
    return  np.array(X_list), np.array(y_list).reshape(-1)

In [10]:
def per_ticker_tt_split(df, target,  size = 0.7, window_size=20):
    Ticker_train_dict = {}
    Ticker_test_dict = {}
    for t in tqdm(df.Ticker.unique()[:10]):
        scaler = MinMaxScaler()
        train_size = df[df['Ticker']==t].iloc[:int(len(df[df.Ticker==t])*size)].drop('Ticker', axis = 1)
        train_size['Datetime'] = pd.to_datetime(train_size['Datetime'])
        train_size.set_index('Datetime', inplace = True)
        scaler.fit(train_size)
        train_size = pd.DataFrame(columns = train_size.columns, data = scaler.transform(train_size))
        test_size = df[df['Ticker']==t].iloc[int(len(df[df.Ticker==t])*size):].drop('Ticker', axis = 1)
        test_size['Datetime'] = pd.to_datetime(test_size['Datetime'])
        test_size.set_index('Datetime', inplace = True)
        test_size = pd.DataFrame(columns = test_size.columns, data = scaler.transform(test_size))

        X_train, y_train = my_window_data(train_size, target, window_size)
        X_test , y_test  = my_window_data(test_size, target, window_size)
        Ticker_train_dict[f'{t}_X_train'], Ticker_train_dict[f'{t}_y_train'] = X_train, y_train
        Ticker_test_dict[f'{t}_X_test'], Ticker_test_dict[f'{t}_y_test'] = X_test , y_test
    return Ticker_train_dict, Ticker_test_dict

In [11]:
train_dict, test_dict = per_ticker_tt_split(df = df_new, target='Adj Close')

100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


In [12]:
import torch
from torch.utils.data import Dataset

class StockDataset(Dataset):
    def __init__(self, Ticker, mode = 'train'):
        self.Ticker = Ticker
        if mode == 'train':
            self.X = torch.tensor(train_dict[f'{Ticker}_X_{mode}']).float()
            self.y = torch.tensor(train_dict[f'{Ticker}_y_{mode}']).float()
        elif mode == 'test':
            self.X = torch.tensor(test_dict[f'{Ticker}_X_{mode}']).float()
            self.y = torch.tensor(test_dict[f'{Ticker}_y_{mode}']).float()
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        return self.X[i], self.y[i]

In [13]:
from torch import nn
class Net(nn.Module):
    # # 기본변수, layer를 초기화해주는 생성자
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers,
                            # dropout = 0.1,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True) 
        
    # 학습 초기화를 위한 함수
    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))
    
    # 예측을 위한 함수
    def forward(self, x):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

In [14]:
feature_num = 6
hidden_dim = 10 
output_dim = 1 
device = 'cpu'

In [15]:
import torch
from torch.utils.data import DataLoader # 데이터로더


In [16]:
def training(epoch, model, train_loader ):
    running_loss = 0

    model.train()
    for idx, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader. dataset)
    return epoch_loss
    
def testing(epoch, model,valid_loader):
    valid_running_loss = 0
    model.eval()
    with torch.no_grad():
        for idx, (X, y) in enumerate(valid_loader):
             X, y = X.to(device), y.to(device)
             y_pred = model(X)
             loss = loss_fn(y_pred, y)
             valid_running_loss += loss.item()
    epoch_valid_loss = valid_running_loss / len(valid_loader.dataset)
    # print(f'epoch:{epoch},\nloss:{round(epoch_loss, 3)},valid_loss:{round(epoch_valid_loss, 3)}')
    return epoch_valid_loss

In [17]:
model = Net(feature_num, hidden_dim, 20, output_dim, 1).to(device)  

In [18]:
ticker_list = list(df_new['Ticker'].unique())

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [20]:
epochs = 100
pbar = tqdm(range(epochs))
total_loss = []
total_val_loss = []
for epoch in pbar:
        loss_list = []
        val_loss_list = []
        for ticker in ticker_list[:10]:
                train_dataset = StockDataset(ticker, mode = 'train')
                valid_dataset = StockDataset(ticker, mode = 'test')
                batch = 20
                loss_fn = nn.MSELoss().to(device)
                train_loader = DataLoader(train_dataset,
                                batch_size = batch,
                                shuffle = False,  
                                drop_last = True)
                valid_loader = DataLoader(valid_dataset,
                                batch_size = batch,
                                shuffle = False,  
                                drop_last = True) 
                
                loss = training(epoch, model, train_loader)
                valid_loss = testing(epoch, model, valid_loader)
                loss_list.append(loss)
                val_loss_list.append(valid_loss)
        print(f'Epoch{str(epoch).zfill(3)}\t\t loss : {round((np.mean(loss_list)), 3)}, valid loss : {round(np.mean(val_loss_list), 3)}')
        total_loss.append(np.mean(loss_list))
        total_val_loss.append(np.mean(val_loss_list))

  0%|          | 0/100 [00:00<?, ?it/s]/Users/junginseo/opt/anaconda3/envs/dl_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [00:04<07:40,  4.66s/it]

Epoch000		 loss : 0.001, valid loss : 0.09


  2%|▏         | 2/100 [00:09<07:31,  4.60s/it]

Epoch001		 loss : 0.0, valid loss : 0.069


  3%|▎         | 3/100 [00:13<07:19,  4.53s/it]

...

Epoch097		 loss : 0.0, valid loss : 0.046


 99%|█████████▉| 99/100 [2:19:35<00:05,  5.62s/it]

Epoch098		 loss : 0.0, valid loss : 0.046


100%|██████████| 100/100 [2:19:40<00:00, 83.81s/it]

Epoch099		 loss : 0.0, valid loss : 0.046
